# Eurovision 2022 (Draft Notebook)


## Potential Research Questions

1. Are Jury Voters less affected by Cultural Influences in Eurovision ?
2. [Needs to be Formulated Well] Does a sympathy-wave effect for Ukraine exist in 2022 Eurovision results?  How can we account for that?


## Plan

### 1. Pre-Analysis or Exploratory Data Analysis

Worth doing again, with newer data.

1. Basically, redo figure 7 from the paper with the newest data

## 2. Agent-Based Models

1. Null Model
2. Model 1
3. Cultural Affinity Model
   a. Cultural Affinity Model part 2


## 3. Find Best Fit

1. Kolmogorov-Smirnov test
2. Hypothesis: Simulated and Empirical FoF come from the same distribution.
    3. This was rejected in the original paper
    
    
### Further Work
1. Consider doing section 5
2. Ambitious: Section 6


Suggestion: Focus on the ABMs as opposed to fully recreating the full paper (K-S tests etc.) 
Also try various things:  subsets of networks.  what happens if you take away a node etc etc.  Give extra weight to Ukraine itself?

In [9]:
import pandas as pd
import numpy as np
import plotly.express as px
import networkx as nx
import matplotlib.pyplot as plt
from mesa import Model, Agent
from mesa.time import RandomActivation
#import pyvis.network as pyvisnetwork
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector


from scipy.stats import rankdata
from scipy.stats import ks_2samp

# 1. Exploratory Data Analysis

First, we load the data for the various years and then visualize the network for a given year

In [10]:
#df = pd.read_excel('https://query.data.world/s/cuokq5sbbqlogvjrdwdeacvp367xnq')
df = pd.read_csv("data/dataworld.csv")
df

,Unnamed: 0,Unnamed: 0.1,Year,(semi-) final,Edition,Jury or Televoting,From country,To country,Points,Duplicate
0,0,0,1975,f,1975f,J,Belgium,Belgium,0,x
1,1,1,1975,f,1975f,J,Belgium,Finland,0,NaN
2,2,2,1975,f,1975f,J,Belgium,France,2,NaN
3,3,3,1975,f,1975f,J,Belgium,Germany,0,NaN
4,4,4,1975,f,1975f,J,Belgium,Ireland,12,NaN
...,...,...,...,...,...,...,...,...,...,...
56307,56307,56307,2022,f,2022f,T,United Kingdom,Serbia,1,NaN
56308,56308,56308,2022,f,2022f,T,United Kingdom,Spain,5,NaN
56309,56309,56309,2022,f,2022f,T,United Kingdom,Sweden,4,NaN
56310,56310,56310,2022,f,2022f,T,United Kingdom,Switzerland,0,NaN


In [11]:
df.rename(columns=lambda x: x.strip(), inplace=True)
df.rename(columns={
    "(semi-) final": "semi_or_final",
    "Jury or Televoting": "jury_or_televoting",
    "From country": "from",
    "To country": "to",
    "Points": "points"
}, inplace=True)

In [12]:
df_finals = df[df["semi_or_final"] == 'f']
df_finals_televoting = df_finals[df_finals["jury_or_televoting"] == "T"]
df_finals_jury = df_finals[df_finals["jury_or_televoting"] == "J"]


display(df_finals_televoting)
display(df_finals_jury)

,Unnamed: 0,Unnamed: 0.1,Year,semi_or_final,Edition,jury_or_televoting,from,to,points,Duplicate
36352,36352,36352,2016,f,2016f,T,Albania,Armenia,2,NaN
36353,36353,36353,2016,f,2016f,T,Albania,Australia,12,NaN
36354,36354,36354,2016,f,2016f,T,Albania,Austria,0,NaN
36355,36355,36355,2016,f,2016f,T,Albania,Azerbaijan,0,NaN
36356,36356,36356,2016,f,2016f,T,Albania,Belgium,0,NaN
...,...,...,...,...,...,...,...,...,...,...
56307,56307,56307,2022,f,2022f,T,United Kingdom,Serbia,1,NaN
56308,56308,56308,2022,f,2022f,T,United Kingdom,Spain,5,NaN
56309,56309,56309,2022,f,2022f,T,United Kingdom,Sweden,4,NaN
56310,56310,56310,2022,f,2022f,T,United Kingdom,Switzerland,0,NaN


,Unnamed: 0,Unnamed: 0.1,Year,semi_or_final,Edition,jury_or_televoting,from,to,points,Duplicate
0,0,0,1975,f,1975f,J,Belgium,Belgium,0,x
1,1,1,1975,f,1975f,J,Belgium,Finland,0,NaN
2,2,2,1975,f,1975f,J,Belgium,France,2,NaN
3,3,3,1975,f,1975f,J,Belgium,Germany,0,NaN
4,4,4,1975,f,1975f,J,Belgium,Ireland,12,NaN
...,...,...,...,...,...,...,...,...,...,...
56283,56283,56283,2022,f,2022f,J,United Kingdom,Serbia,1,NaN
56284,56284,56284,2022,f,2022f,J,United Kingdom,Spain,10,NaN
56285,56285,56285,2022,f,2022f,J,United Kingdom,Sweden,12,NaN
56286,56286,56286,2022,f,2022f,J,United Kingdom,Switzerland,2,NaN


Let's visualize!

# Agent Based Modelling

In [13]:
SCORES = np.array([12, 10, 8, 7, 6, 5, 4, 3, 2, 1])
alpha = 0.1

In [14]:
def calc_All_FoF(df, from_country, points_column = "points", FoF_column = "FoF"):
    Voter_selector = df["from"] == from_country

    voting_countries_2022 = df["from"].unique()


    N = len(voting_countries_2022)
    
    for to_country in df["to"].unique():
        Candidate_selector = df["to"] == to_country

        final_score = sum(df.loc[Candidate_selector, points_column])

        points = df.loc[Voter_selector & Candidate_selector, points_column]

        df.loc[Voter_selector & Candidate_selector, FoF_column ] =  ( points / 12 ) - ( ( final_score - points )/( 12*(N-2) ) )

## Null Model

In this null model, countries freely vote for other countries.  The only restriction is that they can assign only a fixed amount of points

In [15]:
class NullVotingAgent(Agent):
    
    def __init__(self, unique_id, model, df):
        '''
         Create a new Null Voting Agent (A "country")

         Args:
            unique_id: Unique identifier for the agent (A country name)
        '''
        
        super().__init__(unique_id, model)
        
        self.df = df
        
        
    def step(self):
        '''
        Run one step of the agent.
        '''
        
        #print(f"Country {self.unique_id} is now voting")
        
        country_row_selector = self.df["from"] == self.unique_id

        fit = np.random.uniform(0, 1, size=len( self.df[country_row_selector] ))

        points_given = 58 * (fit/sum(fit))
        
        #print(f"{voting_country=}, {points_given=}")
        self.df.loc[country_row_selector, "null_model_points"] += points_given

In [16]:
def get_null_model_ks(model):
    
    data1 = model.df["FoF"]
    data2 = model.df["null_model_FoF"]
    
    return ks_2samp(data1, data2)

class NullVotingModel(Model):
    
    def __init__(self, jury_or_televoting, year):
        '''
        Create a new NullVotingModel model.

         Args:
            jury_or_televoting: a string with values  
                * J: Jury
                * T: Televoting
            year: year of competition
        '''
        
        super().__init__()
        
        df_finals_type = df_finals[df_finals["jury_or_televoting"] == jury_or_televoting]
        df_finals_televoting_year = df_finals_type[df_finals_type["Year"]==year].copy()

        self.df = df_finals_televoting_year
        
        self.datacollector = DataCollector(
            model_reporters={"ks": get_null_model_ks}
        )
        
        self.schedule = RandomActivation(self)
        
        
        self.df["null_model_points"] = 0
        self.df["FoF"] = 0
        self.df["null_model_FoF"] = 0

        voting_countries = self.df["from"].unique()

        for voting_country in voting_countries:
            country_row_selector = self.df["from"] == voting_country
            the_agent = NullVotingAgent(voting_country, self, self.df)
            self.schedule.add(the_agent)
            

    def step(self):
        '''
        Run one step of the model. If All agents are happy, halt the model.
        '''
        
        
        self.schedule.step()
        
        #print("Going to start calc all the FoF")
        for voting_country in self.df["from"].unique():
            calc_All_FoF(self.df, voting_country, 'points', "FoF")        
            calc_All_FoF(self.df, voting_country, 'null_model_points', "null_model_FoF")
            
        self.datacollector.collect(self)

## Model1 ABM
This model also gives points at random, but follows the rules of the contest

In [17]:
class Model1Agent(Agent):
    
    def __init__(self, unique_id, model, df):
        '''
         Create a new Model 1 Voting Agent (A "country")

         Args:
            unique_id: Unique identifier for the agent (A country name)
        '''
        
        super().__init__(unique_id, model)
        self.df = df
        
        
    def step(self):
        '''
        Run one step of the agent.
        '''
        
        country_row_selector = self.df["from"] == self.unique_id

        fit = pd.Series( np.random.uniform(0, 1, size=len(self.df[country_row_selector])) )
        rank = fit.rank(ascending=False)
        points_given = rank.apply(
            lambda x: SCORES[int(x) - 1] if x <= 10 else 0
        )
        
        #print(f"{rank=}, {voting_country=}, {type(points_given)=}")
        self.df.loc[country_row_selector, "model1_points"] += points_given.to_numpy()

In [18]:
def get_model1_ks(model):
    
    data1 = model.df["FoF"]
    data2 = model.df["model1_FoF"]
    
    return ks_2samp(data1, data2)


class Model1Model(Model):
    
    def __init__(self, jury_or_televoting, year):
        '''
        Create a new NullVotingModel model.

         Args:
            jury_or_televoting: a string with values  
                * J: Jury
                * T: Televoting
            year: year of competition
        '''
        
        super().__init__()
        
        df_finals_type = df_finals[df_finals["jury_or_televoting"] == jury_or_televoting]
        df_finals_televoting_year = df_finals_type[df_finals_type["Year"]==year].copy()

        self.df = df_finals_televoting_year
        
        self.datacollector = DataCollector(
            model_reporters={"ks": get_model1_ks}
        )
        
        self.df["model1_points"] = 0
        
        self.schedule = RandomActivation(self)
        

        voting_countries = self.df["from"].unique()

        for voting_country in voting_countries:
            country_row_selector = self.df["from"] == voting_country
            a = Model1Agent(voting_country, self, self.df)
            self.schedule.add(a)
            

    def step(self):
        '''
        Run one step of the model.
        '''
        
        self.schedule.step()
        
        #print("Going to start calc all the FoF")
        for voting_country in self.df["from"].unique():
            calc_All_FoF(self.df, voting_country, 'points', "FoF")        
            calc_All_FoF(self.df, voting_country, 'model1_points', "model1_FoF")
            
        self.datacollector.collect(self)

## Affinity Model

In [19]:
class AffinityModelAgent(Agent):
    
    def __init__(self, unique_id, model, df, alpha, cultural_network):
        '''
         Create a new Model 1 Voting Agent (A "country")

         Args:
            unique_id: Unique identifier for the agent (A country name)
        '''
        
        super().__init__(unique_id, model)
        
        self.df = df
        self.cultural_network = cultural_network
        self.alpha = alpha
        
    def step(self):
        '''
        Run one step of the agent.
        '''
        
        country_row_selector = self.df["from"] == self.unique_id
        
        def affinity_fit(row):
            '''
                row: a dataframe row
            '''
            q = float(self.df["relative_score"].sample(1))

            from_country = row['from']
            to_country = row['to']


            network_part = nx.get_edge_attributes(self.cultural_network, "weight")[(from_country, to_country)]

            the_fit = self.alpha * q + network_part * (1 - self.alpha)

            return the_fit

        
        country_df = self.df[country_row_selector]

        fit = country_df.apply(
            lambda row: affinity_fit(row), axis=1
        )
        #print(f"{unique_id=}, {fit=}")
        rank = fit.rank(ascending=False)
        points_given = rank.apply(
            lambda x: SCORES[int(x) - 1] if x <= 10 else 0
        )
        
        self.df.loc[country_row_selector, "affinity_model_points"] += points_given.to_numpy()


In [20]:
def get_affinity_ks(model):
    
    data1 = model.df["FoF"]
    data2 = model.df["affinity_model_FoF"]
    
    return ks_2samp(data1, data2)


class AffinityModel(Model):
    
    def __init__(self,jury_or_televoting, year, mu, sigma, alpha):
        '''
        Create a new NullVotingModel model.

         Args:
        '''
        
        super().__init__()
        
        self.datacollector = DataCollector(
            model_reporters={"ks": get_affinity_ks}
        )

        
        df_finals_type = df_finals[df_finals["jury_or_televoting"] == jury_or_televoting]
        df_finals_televoting_year = df_finals_type[df_finals_type["Year"]==year].copy()
        self.df = df_finals_televoting_year
        
        self.df["affinity_model_points"] = 0
        
        self.df["relative_score"] = self.df["points"]/sum(self.df["points"])
        
        
        self.cultural_network = nx.from_pandas_edgelist(
            self.df,
            source="from",
            target="to",
            create_using=nx.DiGraph()
        )

        self.mu = mu
        self.sigma = sigma
        self.alpha = alpha

        dict_weight = {}
        for edge in self.cultural_network.edges():
            sampled_weight = np.random.normal(mu, sigma)
            dict_weight[edge] = sampled_weight
        nx.set_edge_attributes(self.cultural_network, values = dict_weight, name = 'weight')
        
        

        voting_countries = self.df["from"].unique()
        
        self.schedule = RandomActivation(self)

        for voting_country in voting_countries:
            a = AffinityModelAgent(voting_country, self, self.df, self.alpha, self.cultural_network)
            self.schedule.add(a)
            

    def step(self):
        '''
        Run one step of the model.
        '''
        
        self.schedule.step()
        
        #print("Going to start calc all the FoF")
        for voting_country in self.df["from"].unique():
            calc_All_FoF(self.df, voting_country, 'points', "FoF")        
            calc_All_FoF(self.df, voting_country, 'affinity_model_points', "affinity_model_FoF")
            
        self.datacollector.collect(self)

# 3. Run the ABMs in a batch for various years



# 4. Friend-Or-Foe for the ABMS

Now that we have Various models and also empirical data, we can calculate the Friend-or-Foe coefficients for each of them

## Batch Run

In [21]:
params_null = {
    "jury_or_televoting": "T",
    "year": [2021, 2022]    # 2020 Contest was cancelled due to COVID-19
}

In [22]:
from mesa.batchrunner import batch_run

#results = batch_run(
#    NullVotingModel,
#    parameters=params_null,
#    iterations=10,
#    max_steps=1,
#    number_processes=None,
#    data_collection_period=-1,
#    display_progress=True,
#)

In [23]:
#res_null = pd.DataFrame(results)
#res_null.to_csv("res_null.csv")

In [24]:
res_null = pd.read_csv("res_null.csv")
display(res_null)

,Unnamed: 0,RunId,iteration,Step,jury_or_televoting,year,ks_statistic,ks_pval
0,0,0,0,1,T,2022,0.207179,1.006719e-18
1,1,1,1,1,T,2022,0.213333,7.782956e-20
2,2,2,0,1,T,2021,0.247976,4.491244e-27
3,3,3,1,1,T,2021,0.255061,1.231384e-28
4,4,4,2,1,T,2021,0.259109,1.503744e-29
5,5,5,2,1,T,2022,0.191795,4.324999e-16
6,6,6,3,1,T,2021,0.230769,1.799179e-23
7,7,7,3,1,T,2022,0.196923,6.045987e-17
8,8,8,4,1,T,2021,0.233806,4.353690e-24
9,9,9,5,1,T,2021,0.258097,2.552057e-29


In [25]:
df_null_2021 = res_null[res_null["year"] == 2021]
df_null_2022 = res_null[res_null["year"] == 2022]

kses_2021 = df_null_2021["ks_statistic"]
kses_2022 = df_null_2022["ks_statistic"]

In [26]:
df_null_2021.loc[kses_2021.idxmin()]

Unnamed: 0                   6
RunId                        6
iteration                    3
Step                         1
jury_or_televoting           T
year                      2021
ks_statistic          0.230769
 ks_pval                   0.0
Name: 6, dtype: object

In [27]:
df_null_2022.loc[kses_2022.idxmin()]

Unnamed: 0                  10
RunId                       10
iteration                    4
Step                         1
jury_or_televoting           T
year                      2022
ks_statistic          0.190769
 ks_pval                   0.0
Name: 10, dtype: object

In [28]:
params_model1 = {
    "jury_or_televoting": "T",
    "year": [2021, 2022]    # 2020 Contest was cancelled due to COVID-19
}

In [29]:
from mesa.batchrunner import batch_run

#results = batch_run(
#    Model1Model,
#    parameters=params_model1,
#    iterations=10,
#    max_steps=1,
#    number_processes=None,
#    data_collection_period=-1,
#    display_progress=True,
#)

In [30]:
#res_model1 = pd.DataFrame(results)
#res_model1.to_csv("res_model1.csv")

In [31]:
res_model1 = pd.read_csv("res_model1.csv")
display(res_model1)

,Unnamed: 0,RunId,iteration,Step,jury_or_televoting,year,ks_statistic,ks_pval
0,0,0,0,1,T,2021,0.381579,1.768730e-64
1,1,1,0,1,T,2022,0.340513,1.717646e-50
2,2,2,1,1,T,2022,0.383590,2.540396e-64
3,3,3,1,1,T,2021,0.375506,2.101635e-62
4,4,4,2,1,T,2021,0.386640,3.092521e-66
5,5,5,2,1,T,2022,0.350769,1.275017e-53
6,6,6,3,1,T,2022,0.351795,6.123565e-54
7,7,7,4,1,T,2022,0.368205,3.565003e-59
8,8,8,3,1,T,2021,0.381579,1.768730e-64
9,9,9,5,1,T,2022,0.366154,1.662541e-58


In [32]:
df_model1_2021 = res_model1[res_model1["year"] == 2021]
df_model1_2022 = res_model1[res_model1["year"] == 2022]

kses_2021 = df_model1_2021["ks_statistic"]
kses_2022 = df_model1_2022["ks_statistic"]

In [33]:
df_model1_2021.loc[kses_2021.idxmin()]

Unnamed: 0                  14
RunId                       14
iteration                    6
Step                         1
jury_or_televoting           T
year                      2021
ks_statistic          0.370445
ks_pval                    0.0
Name: 14, dtype: object

In [34]:
df_model1_2022.loc[kses_2022.idxmin()]

Unnamed: 0                  19
RunId                       19
iteration                    9
Step                         1
jury_or_televoting           T
year                      2022
ks_statistic          0.335385
ks_pval                    0.0
Name: 19, dtype: object

In [35]:
params_affinity = {
    "jury_or_televoting": "T",
    "year": [2021, 2022],    # 2020 Contest was cancelled due to COVID-19
    "mu": np.linspace(-0.1, 0.1, 21),
    "sigma": np.linspace(0, 0.1, 21),
    "alpha": np.linspace(0, 0.5, 3)
} 

In [36]:
from mesa.batchrunner import batch_run
#
#results = batch_run(
#    AffinityModel,
#    parameters=params_affinity,
#    iterations=10,
#    max_steps=1,
#    number_processes=None,
#    data_collection_period=-1,
#    display_progress=True,
#)

In [37]:
#res_affinity = pd.DataFrame(results)
#res_affinity.to_csv("res_affinity.csv")

In [38]:
res_affinity = pd.read_csv("res_affinity.csv")

In [39]:
display(res_affinity)

,Unnamed: 0,RunId,iteration,Step,jury_or_televoting,year,mu,sigma,alpha,ks_statistic,ks_pval
0,0,0,0,1,T,2021,-0.1,0.015,0.25,0.566802,1.003446e-146
1,1,1,0,1,T,2021,-0.1,0.005,0.50,0.535425,4.863312e-130
2,2,2,0,1,T,2021,-0.1,0.005,0.25,0.555668,1.151460e-140
3,3,3,0,1,T,2021,-0.1,0.025,0.00,0.568826,7.638048e-148
4,4,4,0,1,T,2021,-0.1,0.000,0.50,0.404858,8.895390e-73
...,...,...,...,...,...,...,...,...,...,...,...
26455,26455,26455,9,1,T,2022,0.1,0.095,0.00,0.541538,1.899155e-131
26456,26456,26456,9,1,T,2022,0.1,0.100,0.25,0.532308,9.216951e-127
26457,26457,26457,9,1,T,2022,0.1,0.095,0.50,0.560000,3.971871e-141
26458,26458,26458,9,1,T,2022,0.1,0.100,0.00,0.532308,9.216951e-127


In [40]:
df_affinity_2021 = res_affinity[res_affinity["year"] == 2021]
df_affinity_2022 = res_affinity[res_affinity["year"] == 2022]


kses_2021 = df_affinity_2021["ks_statistic"]
kses_2022 = df_affinity_2022["ks_statistic"]

In [41]:
df_affinity_2021.loc[kses_2021.idxmin()]

Unnamed: 0                2900
RunId                     2900
iteration                    1
Step                         1
jury_or_televoting           T
year                      2021
mu                       -0.06
sigma                      0.0
alpha                      0.5
ks_statistic          0.362348
ks_pval                    0.0
Name: 2900, dtype: object

In [42]:
df_affinity_2022.loc[kses_2022.idxmin()]

Unnamed: 0               20791
RunId                    20791
iteration                    7
Step                         1
jury_or_televoting           T
year                      2022
mu                        0.05
sigma                      0.0
alpha                     0.25
ks_statistic          0.348718
ks_pval                    0.0
Name: 20791, dtype: object

## R^2 for the found indices

We now need to get the R^2 for these two indices by comparing FoF and simulated FoF.

In [43]:
from sklearn.metrics import r2_score

R^2 of best Null Model 2021

In [44]:
best_null_model = NullVotingModel(jury_or_televoting="T", year=2021)

best_null_model.step()

In [45]:
best_null_model.df

,Unnamed: 0,Unnamed: 0.1,Year,semi_or_final,Edition,jury_or_televoting,from,to,points,Duplicate,null_model_points,FoF,null_model_FoF
50820,50820,50820,2021,f,2021f,T,Australia,Albania,0,NaN,1.240958,-0.078829,-0.117494
50821,50821,50821,2021,f,2021f,T,Austria,Albania,0,NaN,2.563725,-0.078829,-0.004284
50822,50822,50822,2021,f,2021f,T,Azerbaijan,Albania,0,NaN,2.870115,-0.078829,0.021938
50823,50823,50823,2021,f,2021f,T,Belgium,Albania,0,NaN,3.114643,-0.078829,0.042866
50824,50824,50824,2021,f,2021f,T,Bulgaria,Albania,0,NaN,4.755444,-0.078829,0.183295
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51803,51803,51803,2021,f,2021f,T,Spain,United Kingdom,0,NaN,3.273280,0.000000,0.080025
51804,51804,51804,2021,f,2021f,T,Sweden,United Kingdom,0,NaN,2.586689,0.000000,0.021263
51805,51805,51805,2021,f,2021f,T,Switzerland,United Kingdom,0,NaN,0.768029,0.000000,-0.134388
51806,51806,51806,2021,f,2021f,T,The Netherlands,United Kingdom,0,NaN,0.822677,0.000000,-0.129711


In [46]:
FoF_true = best_null_model.df["FoF"]
FoF_pred = best_null_model.df["null_model_FoF"]

r2_score(FoF_true, FoF_pred)

-0.2578926752384363

R^2 of best Null Model 2022

In [63]:
best_null_model = NullVotingModel(jury_or_televoting="T", year=2022)

best_null_model.step()

In [64]:
best_null_model.df

,Unnamed: 0,Unnamed: 0.1,Year,semi_or_final,Edition,jury_or_televoting,from,to,points,Duplicate,null_model_points,FoF,null_model_FoF
54387,54387,54387,2022,f,2022f,T,Albania,Armenia,0,NaN,0.633757,-0.046053,-0.142262
54388,54388,54388,2022,f,2022f,T,Albania,Australia,0,NaN,2.455183,-0.004386,0.046011
54389,54389,54389,2022,f,2022f,T,Albania,Azerbaijan,0,NaN,0.947754,-0.006579,-0.113455
54390,54390,54390,2022,f,2022f,T,Albania,Belgium,0,NaN,1.856861,-0.010965,-0.019653
54391,54391,54391,2022,f,2022f,T,Albania,Czech Republic,0,NaN,1.633645,-0.010965,-0.078506
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56307,56307,56307,2022,f,2022f,T,United Kingdom,Serbia,1,NaN,0.243919,-0.407895,-0.206721
56308,56308,56308,2022,f,2022f,T,United Kingdom,Spain,5,NaN,3.857890,-0.050439,0.111350
56309,56309,56309,2022,f,2022f,T,United Kingdom,Sweden,4,NaN,3.273740,-0.050439,0.083532
56310,56310,56310,2022,f,2022f,T,United Kingdom,Switzerland,0,NaN,0.931384,0.000000,-0.113684


In [65]:
FoF_true = best_null_model.df["FoF"]
FoF_pred = best_null_model.df["null_model_FoF"]

r2_score(FoF_true, FoF_pred)

-0.31812641018011356

R^2 of best Model1 2021

In [66]:
best_model1_model = Model1Model(jury_or_televoting="T", year=2021)

best_model1_model.step()

In [67]:
best_model1_model.df

,Unnamed: 0,Unnamed: 0.1,Year,semi_or_final,Edition,jury_or_televoting,from,to,points,Duplicate,model1_points,FoF,model1_FoF
50820,50820,50820,2021,f,2021f,T,Australia,Albania,0,NaN,2,-0.078829,-0.027027
50821,50821,50821,2021,f,2021f,T,Austria,Albania,0,NaN,0,-0.078829,-0.198198
50822,50822,50822,2021,f,2021f,T,Azerbaijan,Albania,0,NaN,12,-0.078829,0.828829
50823,50823,50823,2021,f,2021f,T,Belgium,Albania,0,NaN,5,-0.078829,0.229730
50824,50824,50824,2021,f,2021f,T,Bulgaria,Albania,0,NaN,6,-0.078829,0.315315
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51803,51803,51803,2021,f,2021f,T,Spain,United Kingdom,0,NaN,12,0.000000,0.871622
51804,51804,51804,2021,f,2021f,T,Sweden,United Kingdom,0,NaN,0,0.000000,-0.155405
51805,51805,51805,2021,f,2021f,T,Switzerland,United Kingdom,0,NaN,0,0.000000,-0.155405
51806,51806,51806,2021,f,2021f,T,The Netherlands,United Kingdom,0,NaN,0,0.000000,-0.155405


In [68]:
FoF_true = best_model1_model.df["FoF"]
FoF_pred = best_model1_model.df["model1_FoF"]

r2_score(FoF_true, FoF_pred)

-1.678316554097107

R^2 of best Model1 2022

In [69]:
best_model1_model = Model1Model(jury_or_televoting="T", year=2022)

best_model1_model.step()

In [70]:
best_model1_model.df

,Unnamed: 0,Unnamed: 0.1,Year,semi_or_final,Edition,jury_or_televoting,from,to,points,Duplicate,model1_points,FoF,model1_FoF
54387,54387,54387,2022,f,2022f,T,Albania,Armenia,0,NaN,0,-0.046053,-0.173246
54388,54388,54388,2022,f,2022f,T,Albania,Australia,0,NaN,2,-0.004386,0.017544
54389,54389,54389,2022,f,2022f,T,Albania,Azerbaijan,0,NaN,1,-0.006579,-0.028509
54390,54390,54390,2022,f,2022f,T,Albania,Belgium,0,NaN,6,-0.010965,0.228070
54391,54391,54391,2022,f,2022f,T,Albania,Czech Republic,0,NaN,0,-0.010965,-0.287281
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56307,56307,56307,2022,f,2022f,T,United Kingdom,Serbia,1,NaN,0,-0.407895,-0.081140
56308,56308,56308,2022,f,2022f,T,United Kingdom,Spain,5,NaN,0,-0.050439,-0.175439
56309,56309,56309,2022,f,2022f,T,United Kingdom,Sweden,4,NaN,0,-0.050439,-0.157895
56310,56310,56310,2022,f,2022f,T,United Kingdom,Switzerland,0,NaN,5,0.000000,0.210526


In [71]:
FoF_true = best_model1_model.df["FoF"]
FoF_pred = best_model1_model.df["model1_FoF"]

r2_score(FoF_true, FoF_pred)

-1.9900067062746172

R^2 of best Affinity Model 2021

In [72]:
best_affinity_model = AffinityModel(jury_or_televoting="T", year=2021, mu=-0.06, sigma=0.0, alpha=0.5)

best_affinity_model.step()

In [73]:
best_affinity_model.df

,Unnamed: 0,Unnamed: 0.1,Year,semi_or_final,Edition,jury_or_televoting,from,to,points,Duplicate,affinity_model_points,relative_score,FoF,affinity_model_FoF
50820,50820,50820,2021,f,2021f,T,Australia,Albania,0,NaN,0,0.0,-0.078829,-0.211712
50821,50821,50821,2021,f,2021f,T,Austria,Albania,0,NaN,1,0.0,-0.078829,-0.126126
50822,50822,50822,2021,f,2021f,T,Azerbaijan,Albania,0,NaN,10,0.0,-0.078829,0.644144
50823,50823,50823,2021,f,2021f,T,Belgium,Albania,0,NaN,0,0.0,-0.078829,-0.211712
50824,50824,50824,2021,f,2021f,T,Bulgaria,Albania,0,NaN,0,0.0,-0.078829,-0.211712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51803,51803,51803,2021,f,2021f,T,Spain,United Kingdom,0,NaN,0,0.0,0.000000,-0.204955
51804,51804,51804,2021,f,2021f,T,Sweden,United Kingdom,0,NaN,12,0.0,0.000000,0.822072
51805,51805,51805,2021,f,2021f,T,Switzerland,United Kingdom,0,NaN,0,0.0,0.000000,-0.204955
51806,51806,51806,2021,f,2021f,T,The Netherlands,United Kingdom,0,NaN,0,0.0,0.000000,-0.204955


In [74]:
FoF_true = best_affinity_model.df["FoF"]
FoF_pred = best_affinity_model.df["affinity_model_FoF"]

r2_score(FoF_true, FoF_pred)

-1.8531723907225688

R^2 of best Affinity Model 2022

In [75]:
best_affinity_model = AffinityModel(jury_or_televoting="T", year=2022, mu=0.05, sigma=0.0, alpha=0.25)

best_affinity_model.step()
best_affinity_model.df

,Unnamed: 0,Unnamed: 0.1,Year,semi_or_final,Edition,jury_or_televoting,from,to,points,Duplicate,affinity_model_points,relative_score,FoF,affinity_model_FoF
54387,54387,54387,2022,f,2022f,T,Albania,Armenia,0,NaN,0,0.000000,-0.046053,-0.236842
54388,54388,54388,2022,f,2022f,T,Albania,Australia,0,NaN,12,0.000000,-0.004386,0.813596
54389,54389,54389,2022,f,2022f,T,Albania,Azerbaijan,0,NaN,0,0.000000,-0.006579,-0.179825
54390,54390,54390,2022,f,2022f,T,Albania,Belgium,0,NaN,0,0.000000,-0.010965,-0.239035
54391,54391,54391,2022,f,2022f,T,Albania,Czech Republic,0,NaN,0,0.000000,-0.010965,-0.223684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56307,56307,56307,2022,f,2022f,T,United Kingdom,Serbia,1,NaN,0,0.000442,-0.407895,-0.188596
56308,56308,56308,2022,f,2022f,T,United Kingdom,Spain,5,NaN,0,0.002210,-0.050439,-0.122807
56309,56309,56309,2022,f,2022f,T,United Kingdom,Sweden,4,NaN,6,0.001768,-0.050439,0.265351
56310,56310,56310,2022,f,2022f,T,United Kingdom,Switzerland,0,NaN,8,0.000000,0.000000,0.482456


In [76]:
FoF_true = best_affinity_model.df["FoF"]
FoF_pred = best_affinity_model.df["affinity_model_FoF"]

r2_score(FoF_true, FoF_pred)

-2.4393459965429627